## Imports

### Import Functions SQL (PySpark)

In [0]:
# Import SQL Functions
import pyspark.sql.functions as fn

from pyspark.sql.functions import desc, asc
from pyspark.sql.functions import when
from pyspark.sql.functions import *

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

## Conectando

In [0]:
# Utilizando a Access Key da Storage Account

spark.conf.set("fs.azure.account.key.saneobanklucas.dfs.core.windows.net", 
               "vwzvt5OqOoUhdCK62Jou9tF67bGU38Qrj4FgjzC3zQSS2hSxESla14QU+PXuxk+pLd1+k3Ym2dKd+AStbY19lQ=="
               )

## Extraindo os Arquivos (fonte / destino)

In [0]:
path_bronze = "abfss://bronze@saneobanklucas.dfs.core.windows.net/"
path_gold   = "abfss://gold@saneobanklucas.dfs.core.windows.net/"

dfClientSRC = (spark
               .read
               .format("csv")
               .option("delimiter", ",")
               .option("header", "true")
               .option("inferschema", "true")
               .load(path_bronze + 'NeoBank_Modelling_Source.csv')
               )

dfClientTAG = (spark
               .read
               .format("csv")
               .option("delimiter", ",")
               .option("header", "true")
               .option("inferschema", "true")
               .load(path_gold + 'NeoBank_Modelling.csv')
               )

In [0]:
dfClientSRC.display()
dfClientTAG.limit(10).display()

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
10001,15628319,Jack da Silva - MAJ,792,Canada,Male,28,4,86555.79,1,1,1,45000.78,0
10001,215628319,New Cliente,500,Canada,Male,28,4,86555.79,1,1,1,45000.78,0


RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.8,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0
6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
7,15592531,Bartlett,822,France,Male,50,7,0.0,2,1,1,10062.8,0
8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.5,0
10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


## Realizando Transformações

### Filtro de Ativos

In [0]:
dfClientSRC = (dfClientSRC
               .drop_duplicates()
               .filter(fn.col('IsActiveMember') == 1)
               )

dfClientSRC.display()

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
10001,215628319,New Cliente,500,Canada,Male,28,4,86555.79,1,1,1,45000.78,0
10001,15628319,Jack da Silva - MAJ,792,Canada,Male,28,4,86555.79,1,1,1,45000.78,0


### Renomear Colunas

In [0]:
# Renomear Source
dfClientSRC = (dfClientSRC
               .withColumnRenamed('RowNumber', 'RowNumber_SRC')
               .withColumnRenamed('CustomerId', 'CustomerId_SRC')
               .withColumnRenamed('Surname', 'Surname_SRC')
               .withColumnRenamed('CreditScore', 'CreditScore_SRC')
               .withColumnRenamed('Geography', 'Geography_SRC')
               .withColumnRenamed('Gender', 'Gender_SRC')
               .withColumnRenamed('Age', 'Age_SRC')
               .withColumnRenamed('Tenure', 'Tenure_SRC')
               .withColumnRenamed('Balance', 'Balance_SRC')
               .withColumnRenamed('NumOfProducts', 'NumOfProducts_SRC')
               .withColumnRenamed('HasCrCard', 'HasCrCard_SRC')
               .withColumnRenamed('IsActiveMember', 'IsActiveMember_SRC')
               .withColumnRenamed('EstimatedSalary', 'EstimatedSalary_SRC')
               .withColumnRenamed('Exited', 'Exited_SRC')
               )

dfClientTAG = (dfClientTAG
               .withColumnRenamed('RowNumber', 'RowNumber_TAG')
               .withColumnRenamed('CustomerId', 'CustomerId_TAG')
               .withColumnRenamed('Surname', 'Surname_TAG')
               .withColumnRenamed('CreditScore', 'CreditScore_TAG')
               .withColumnRenamed('Geography', 'Geography_TAG')
               .withColumnRenamed('Gender', 'Gender_TAG')
               .withColumnRenamed('Age', 'Age_TAG')
               .withColumnRenamed('Tenure', 'Tenure_TAG')
               .withColumnRenamed('Balance', 'Balance_TAG')
               .withColumnRenamed('NumOfProducts', 'NumOfProducts_TAG')
               .withColumnRenamed('HasCrCard', 'HasCrCard_TAG')
               .withColumnRenamed('IsActiveMember', 'IsActiveMember_TAG')
               .withColumnRenamed('EstimatedSalary', 'EstimatedSalary_TAG')
               .withColumnRenamed('Exited', 'Exited_TAG')
               )

dfClientSRC.display()
dfClientTAG.limit(10).display()

RowNumber_SRC,CustomerId_SRC,Surname_SRC,CreditScore_SRC,Geography_SRC,Gender_SRC,Age_SRC,Tenure_SRC,Balance_SRC,NumOfProducts_SRC,HasCrCard_SRC,IsActiveMember_SRC,EstimatedSalary_SRC,Exited_SRC
10001,215628319,New Cliente,500,Canada,Male,28,4,86555.79,1,1,1,45000.78,0
10001,15628319,Jack da Silva - MAJ,792,Canada,Male,28,4,86555.79,1,1,1,45000.78,0


RowNumber_TAG,CustomerId_TAG,Surname_TAG,CreditScore_TAG,Geography_TAG,Gender_TAG,Age_TAG,Tenure_TAG,Balance_TAG,NumOfProducts_TAG,HasCrCard_TAG,IsActiveMember_TAG,EstimatedSalary_TAG,Exited_TAG
1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.8,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0
6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
7,15592531,Bartlett,822,France,Male,50,7,0.0,2,1,1,10062.8,0
8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.5,0
10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


### Join Source e TAG

In [0]:
dfClientJoin = dfClientSRC.join(dfClientTAG, dfClientSRC.CustomerId_SRC == dfClientTAG.CustomerId_TAG, 'left')

dfClientJoin.display()

RowNumber_SRC,CustomerId_SRC,Surname_SRC,CreditScore_SRC,Geography_SRC,Gender_SRC,Age_SRC,Tenure_SRC,Balance_SRC,NumOfProducts_SRC,HasCrCard_SRC,IsActiveMember_SRC,EstimatedSalary_SRC,Exited_SRC,RowNumber_TAG,CustomerId_TAG,Surname_TAG,CreditScore_TAG,Geography_TAG,Gender_TAG,Age_TAG,Tenure_TAG,Balance_TAG,NumOfProducts_TAG,HasCrCard_TAG,IsActiveMember_TAG,EstimatedSalary_TAG,Exited_TAG
10001,215628319,New Cliente,500,Canada,Male,28,4,86555.79,1,1,1,45000.78,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
10001,15628319,Jack da Silva - MAJ,792,Canada,Male,28,4,86555.79,1,1,1,45000.78,0,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,1,38190.78,0


### Novos Clientes

In [0]:
dfNewRegister = dfClientJoin.where(fn.col('CustomerId_TAG').isNull())
dfNewRegister = dfNewRegister.select(dfClientSRC.columns)
dfNewRegister.display()

RowNumber_SRC,CustomerId_SRC,Surname_SRC,CreditScore_SRC,Geography_SRC,Gender_SRC,Age_SRC,Tenure_SRC,Balance_SRC,NumOfProducts_SRC,HasCrCard_SRC,IsActiveMember_SRC,EstimatedSalary_SRC,Exited_SRC
10001,215628319,New Cliente,500,Canada,Male,28,4,86555.79,1,1,1,45000.78,0


### Update

In [0]:
dfUpdate = dfClientJoin.where("""(CustomerId_SRC == CustomerId_TAG) or 
                              (Surname_SRC != Surname_TAG) or 
                              (CreditScore_SRC != CreditScore_TAG) or 
                              (Geography_SRC != Geography_TAG) or 
                              (Gender_SRC != Gender_TAG) or 
                              (Age_SRC != Age_TAG) or 
                              (Tenure_SRC != Tenure_TAG) or 
                              (Balance_SRC != Balance_TAG) or
                              (NumOfProducts_SRC != NumOfProducts_TAG) or 
                              (HasCrCard_SRC != HasCrCard_TAG) or 
                              (IsActiveMember_SRC != IsActiveMember_TAG) or 
                              (EstimatedSalary_SRC != EstimatedSalary_TAG) or 
                              (Exited_SRC != Exited_TAG)""")

# Selecionar DF de Updates Ativos
dfUpdateAtivos = dfUpdate.select(dfClientSRC.columns)

# Selecionar DF de Updates Inativos
dfUpdateInativos = dfUpdate.select(dfClientTAG.columns)

dfUpdateAtivos.display()
dfUpdateInativos.display()

RowNumber_SRC,CustomerId_SRC,Surname_SRC,CreditScore_SRC,Geography_SRC,Gender_SRC,Age_SRC,Tenure_SRC,Balance_SRC,NumOfProducts_SRC,HasCrCard_SRC,IsActiveMember_SRC,EstimatedSalary_SRC,Exited_SRC
10001,15628319,Jack da Silva - MAJ,792,Canada,Male,28,4,86555.79,1,1,1,45000.78,0


RowNumber_TAG,CustomerId_TAG,Surname_TAG,CreditScore_TAG,Geography_TAG,Gender_TAG,Age_TAG,Tenure_TAG,Balance_TAG,NumOfProducts_TAG,HasCrCard_TAG,IsActiveMember_TAG,EstimatedSalary_TAG,Exited_TAG
10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,1,38190.78,0


### Inativar Update

In [0]:
dfUpdateInativos = (dfUpdateInativos
                    .select('*')
                    .withColumn('IsActiveMember_TAG', when(dfUpdateInativos.IsActiveMember_TAG == 1, 0)
                                                      .otherwise(dfUpdateInativos.IsActiveMember_TAG)
                                )
                    )

dfUpdateInativos.show()

+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+
RowNumber_TAG|CustomerId_TAG|Surname_TAG|CreditScore_TAG|Geography_TAG|Gender_TAG|Age_TAG|Tenure_TAG|Balance_TAG|NumOfProducts_TAG|HasCrCard_TAG|IsActiveMember_TAG|EstimatedSalary_TAG|Exited_TAG|
+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+
 10000| 15628319| Walker| 792| France| Female| 28| 4| 130142.79| 1| 1| 0| 38190.78| 0|
+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+

### Dataframes: dfNewRegister, dfUpdate, dfUpdateInativos

In [0]:
dfNewRegister.display()
dfUpdateAtivos.display()
dfUpdateInativos.display()

RowNumber_SRC,CustomerId_SRC,Surname_SRC,CreditScore_SRC,Geography_SRC,Gender_SRC,Age_SRC,Tenure_SRC,Balance_SRC,NumOfProducts_SRC,HasCrCard_SRC,IsActiveMember_SRC,EstimatedSalary_SRC,Exited_SRC
10001,215628319,New Cliente,500,Canada,Male,28,4,86555.79,1,1,1,45000.78,0


RowNumber_SRC,CustomerId_SRC,Surname_SRC,CreditScore_SRC,Geography_SRC,Gender_SRC,Age_SRC,Tenure_SRC,Balance_SRC,NumOfProducts_SRC,HasCrCard_SRC,IsActiveMember_SRC,EstimatedSalary_SRC,Exited_SRC
10001,15628319,Jack da Silva - MAJ,792,Canada,Male,28,4,86555.79,1,1,1,45000.78,0


RowNumber_TAG,CustomerId_TAG,Surname_TAG,CreditScore_TAG,Geography_TAG,Gender_TAG,Age_TAG,Tenure_TAG,Balance_TAG,NumOfProducts_TAG,HasCrCard_TAG,IsActiveMember_TAG,EstimatedSalary_TAG,Exited_TAG
10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


## Load

### Inativar o registro

In [0]:
dfmerge = dfClientTAG.union(dfUpdateInativos)

dfmerge.where(fn.col('CustomerId_TAG') == 15628319).show()
dfmerge.withColumn('_row_number', row_number().over(Window.partitionBy(fn.col('CustomerId_TAG')).orderBy(fn.col('IsActiveMember_TAG')))).where(fn.col('CustomerId_TAG') == 15628319).show()
dfmerge.count()

+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+
RowNumber_TAG|CustomerId_TAG|Surname_TAG|CreditScore_TAG|Geography_TAG|Gender_TAG|Age_TAG|Tenure_TAG|Balance_TAG|NumOfProducts_TAG|HasCrCard_TAG|IsActiveMember_TAG|EstimatedSalary_TAG|Exited_TAG|
+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+
 10000| 15628319| Walker| 792| France| Female| 28| 4| 130142.79| 1| 1| 1| 38190.78| 0|
 10000| 15628319| Walker| 792| France| Female| 28| 4| 130142.79| 1| 1| 0| 38190.78| 0|
+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+

+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+-----------+
RowNumber_TAG|CustomerId_TAG|Surname_TAG|CreditScore_TAG|Geography_TAG|Gender_TAG|Age_TAG|Tenure_TAG|Balance_TAG|NumOfProducts_TAG|HasCrCard_TAG|IsActiveMember_TAG|EstimatedSalary_TAG|Exited_TAG|_row_number|
+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+-----------+
 10000| 15628319| Walker| 792| France| Female| 28| 4| 130142.79| 1| 1| 0| 38190.78| 0| 1|
 10000| 15628319| Walker| 792| France| Female| 28| 4| 130142.79| 1| 1| 1| 38190.78| 0| 2|
+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+-----------+

Out[12]: 10001

In [0]:
dfmerge = (dfmerge
           .withColumn('_row_number', row_number()
                                      .over(Window.partitionBy(fn.col('CustomerId_TAG'))
                                            .orderBy(fn.col('IsActiveMember_TAG')))
                       ).where(fn.col('_row_number') == 1)
           .drop(fn.col('_row_number'))
           )

dfmerge.show(10, False)

+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+
RowNumber_TAG|CustomerId_TAG|Surname_TAG|CreditScore_TAG|Geography_TAG|Gender_TAG|Age_TAG|Tenure_TAG|Balance_TAG|NumOfProducts_TAG|HasCrCard_TAG|IsActiveMember_TAG|EstimatedSalary_TAG|Exited_TAG|
+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+
1288 |15565701 |Ferri |698 |Spain |Female |39 |9 |161993.89 |1 |0 |0 |90212.38 |0 |
4199 |15565706 |Akobundu |612 |Spain |Male |35 |1 |0.0 |1 |1 |1 |83256.26 |1 |
7091 |15565714 |Cattaneo |601 |France |Male |47 |1 |64430.06 |2 |0 |1 |96517.97 |0 |
2021 |15565779 |Kent |627 |Germany |Female |30 |6 |57809.32 |1 |1 |0 |188258.49 |0 |
3698 |15565796 |Docherty |745 |Germany |Male |48 |10 |96048.55 |1 |1 |0 |74510.65 |0 |
3417 |15565806 |Toosey |532 |France |Male |38 |9 |0.0 |2 |0 |0 |30583.95 |0 |
6882 |15565878 |Bates |631 |Spain |Male |29 |3 |0.0 |2 |1 |1 |197963.46 |0 |
9515 |15565879 |Riley |845 |France |Female |28 |9 |0.0 |2 |1 |1 |56185.98 |0 |
5796 |15565891 |Dipietro |709 |France |Male |39 |8 |0.0 |2 |1 |0 |56214.09 |0 |
7187 |15565996 |Arnold |653 |France |Male |44 |8 |0.0 |2 |1 |1 |154639.72 |0 |
+-------------+--------------+-----------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+
only showing top 10 rows

### Acrescentar novos registros + Updates

In [0]:
dfversionfinal = dfmerge.union(dfNewRegister).union(dfUpdateAtivos)
dfversionfinal.limit(10).display()
dfversionfinal.count()

RowNumber_TAG,CustomerId_TAG,Surname_TAG,CreditScore_TAG,Geography_TAG,Gender_TAG,Age_TAG,Tenure_TAG,Balance_TAG,NumOfProducts_TAG,HasCrCard_TAG,IsActiveMember_TAG,EstimatedSalary_TAG,Exited_TAG
1288,15565701,Ferri,698,Spain,Female,39,9,161993.89,1,0,0,90212.38,0
4199,15565706,Akobundu,612,Spain,Male,35,1,0.0,1,1,1,83256.26,1
7091,15565714,Cattaneo,601,France,Male,47,1,64430.06,2,0,1,96517.97,0
2021,15565779,Kent,627,Germany,Female,30,6,57809.32,1,1,0,188258.49,0
3698,15565796,Docherty,745,Germany,Male,48,10,96048.55,1,1,0,74510.65,0
3417,15565806,Toosey,532,France,Male,38,9,0.0,2,0,0,30583.95,0
6882,15565878,Bates,631,Spain,Male,29,3,0.0,2,1,1,197963.46,0
9515,15565879,Riley,845,France,Female,28,9,0.0,2,1,1,56185.98,0
5796,15565891,Dipietro,709,France,Male,39,8,0.0,2,1,0,56214.09,0
7187,15565996,Arnold,653,France,Male,44,8,0.0,2,1,1,154639.72,0


Out[14]: 10002

In [0]:
dfversionfinal.where((fn.col('CustomerId_TAG') == 15628319) | (fn.col('CustomerId_TAG') == 215628319)).show()

+-------------+--------------+-------------------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+
RowNumber_TAG|CustomerId_TAG| Surname_TAG|CreditScore_TAG|Geography_TAG|Gender_TAG|Age_TAG|Tenure_TAG|Balance_TAG|NumOfProducts_TAG|HasCrCard_TAG|IsActiveMember_TAG|EstimatedSalary_TAG|Exited_TAG|
+-------------+--------------+-------------------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+
 10000| 15628319| Walker| 792| France| Female| 28| 4| 130142.79| 1| 1| 0| 38190.78| 0|
 10001| 215628319| New Cliente| 500| Canada| Male| 28| 4| 86555.79| 1| 1| 1| 45000.78| 0|
 10001| 15628319|Jack da Silva - MAJ| 792| Canada| Male| 28| 4| 86555.79| 1| 1| 1| 45000.78| 0|
+-------------+--------------+-------------------+---------------+-------------+----------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+----------+

### Renomeando as Colunas Corretamente

In [0]:
dfversionfinal = (dfversionfinal
                  .withColumnRenamed('RowNumber_TAG', 'RowNumber')
                  .withColumnRenamed('CustomerId_TAG', 'CustomerId')
                  .withColumnRenamed('Surname_TAG', 'Surname')
                  .withColumnRenamed('CreditScore_TAG', 'CreditScore')
                  .withColumnRenamed('Geography_TAG', 'Geography')
                  .withColumnRenamed('Gender_TAG', 'Gender')
                  .withColumnRenamed('Age_TAG', 'Age')
                  .withColumnRenamed('Tenure_TAG', 'Tenure')
                  .withColumnRenamed('Balance_TAG', 'Balance')
                  .withColumnRenamed('NumOfProducts_TAG', 'NumOfProducts')
                  .withColumnRenamed('HasCrCard_TAG', 'HasCrCard')
                  .withColumnRenamed('IsActiveMember_TAG', 'IsActiveMember')
                  .withColumnRenamed('EstimatedSalary_TAG', 'EstimatedSalary')
                  .withColumnRenamed('Exited_TAG', 'Exited')
                  )

dfversionfinal.columns

Out[16]: ['RowNumber',
 'CustomerId',
 'Surname',
 'CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

### Load Final

In [0]:
path_silver = "abfss://silver@saneobanklucas.dfs.core.windows.net/"

# Deixa apenas uma partição do arquivo Parquet
# dfversionfinal.coalesce(1).write.mode('overwrite').parquet(path_silver + 'NeoBank_Modelling/Parquet')

# Não especifica a quantidade de partições Parquet
dfversionfinal.write.mode('overwrite').parquet(path_silver + 'NeoBank_Modelling/Parquet')